In [25]:
import numpy as np
import pandas as pd
import os
import sys
import random
import matplotlib.pyplot as plt
import operator
import seaborn as sns
import pymongo
from pymongo import MongoClient
import time
import re

import _annoncfg_

%matplotlib inline

In [15]:
host = _annoncfg_.DBCFG['ANNONCFG']['host']
port_number = _annoncfg_.DBCFG['ANNONCFG']['port']

In [18]:
# client = MongoClient(host, port_number)
client = MongoClient('10.4.71.69', port_number)

In [20]:
db_names = client.list_database_names()

In [49]:
client.list_database_names()

['PXL-081119_174857',
 'PXL-081119_181309',
 'PXL-081119_184710',
 'PXL-081119_193452',
 'PXL-081119_215023',
 'PXL-081119_225515',
 'PXL-081119_231523',
 'PXL-081119_234430',
 'PXL-091119_011424',
 'PXL-091119_013820',
 'PXL-091119_014238',
 'PXL-091119_032434',
 'PXL-091119_051633',
 'PXL-091119_061417',
 'PXL-091119_062639',
 'PXL-091119_064318',
 'PXL-091119_064443',
 'PXL-091119_065015',
 'PXL-091119_065353',
 'PXL-091119_065953',
 'PXL-091119_070646',
 'PXL-091119_071125',
 'PXL-091119_073030',
 'PXL-091119_073506',
 'PXL-091119_073608',
 'PXL-091119_073716',
 'PXL-131119_190820',
 'PXL-131119_191127',
 'PXL-141119_185550',
 'PXL-151119_175327',
 'PXL-161019_175838',
 'PXL-171019_185702',
 'PXL-281119_140106',
 'PXL-281119_144018',
 'PXL-281119_144432',
 'PXL-281119_154739',
 'PXL-291119_180404',
 'admin',
 'annon',
 'annon_v3',
 'annon_v4',
 'annon_v5',
 'annon_v6',
 'annon_v7',
 'annon_v8',
 'annon_v9',
 'config',
 'local']

In [39]:
# aids = []
# for i in db_names:
#     if re.match(regex, i):
#         aids.append(i)
regex = r".*[\w-]+[\d{6}]_.*[\d{6}]"
aids = [i for i in db_names if re.match(regex, i)]

In [79]:
db = client[x]

In [78]:
db.list_collection_names()

['AIDS', 'EVALUATE', 'ANNOTATIONS', 'IMAGES', 'STATS', 'CLASSINFO']

In [81]:
aids_collection = db['AIDS']

In [83]:
stat = aids_collection.find()
stat_j = list(stat)

In [86]:
df = pd.DataFrame(stat_j)

In [90]:
df

,_id,created_on,allowed_file_type,allowed_image_type,allowed_video_type,anndb_id,anndb_rel_id,annon_type,class_ids,class_map,...,rel_type,report,return_hmd,splits,stats,summary,test_mode,timestamp,train,train_mode
0,5dc5761b3cdebfca89925324,2019-11-08 19:35:15 +05:30,"[.txt, .csv, .yml, .json]","[.pdf, .png, .jpg, .jpeg, .gif]",[.mp4],PXL-081119_193452,None,hmd,None,None,...,aids,[],None,"[train, val, test]","{'train': {'labels': ['traffic_sign'], 'classi...","{'total_images': 15838, 'total_annotations': 2...",inference,081119_193452,[],training


In [101]:
df['stats'][0]['train']

{'labels': ['traffic_sign'],
 'classinfo': [{'lbl_id': 'traffic_sign',
   'name': 'traffic_sign',
   'source': 'hmd'}],
 'total_labels': 1,
 'total_annotations': 18860,
 'total_images': 11878,
 'total_unique_images': 0,
 'annotation_per_img': [],
 'label_per_img': [],
 'maskarea': [],
 'bboxarea': [],
 'colors': None,
 'dbid': 'aids-19dac9af-ca17-473b-b611-65dffb7a614e',
 'timestamp': '081119_193452',
 'subset': 'train',
 '_id': ObjectId('5dc5761598e23cd7f6abc6d2')}